**Подготовка** **данных**

In [175]:
pip install biopython

In [176]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/chr1.fa.gz

--2021-12-01 17:23:29--  http://hgdownload.cse.ucsc.edu/goldenPath/hg19/chromosomes/chr1.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73773666 (70M) [application/x-gzip]
Saving to: ‘chr1.fa.gz’

chr1.fa.gz          100%[===================>]  70.36M  27.1MB/s    in 2.6s    

2021-12-01 17:23:32 (27.1 MB/s) - ‘chr1.fa.gz’ saved [73773666/73773666]



In [177]:
!gunzip chr1.fa.gz

gzip: chr1.fa already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [178]:
!tail chr1.fa

NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNN


In [179]:
!ls -lah chr1.fa

-rw-r--r-- 1 root root 243M Mar 20  2009 chr1.fa


In [180]:
import re
from Bio import SeqIO
import pandas as pd
import numpy as np
import random

In [181]:
input_file = 'chr1.fa'

Подгружаем наш файлик с пересечением.

In [182]:
G4ChIP ='H2AFZ_A549.intersect_with_DeepZ.bed'

In [183]:
chip=pd.read_csv(G4ChIP, sep="\t",names=['chr', 'start', 'end'])
chip

,chr,start,end
0,chr1,762616,762898
1,chr1,840032,840251
2,chr1,894548,894648
3,chr1,894839,894874
4,chr1,1004735,1004904
...,...,...,...
13203,chrX,154299738,154299918
13204,chrY,2709529,2709555
13205,chrY,2709577,2709618
13206,chrY,2709789,2709805


In [184]:
chip['len'] = chip['end'] - chip['start'] + 1
chip

,chr,start,end,len
0,chr1,762616,762898,283
1,chr1,840032,840251,220
2,chr1,894548,894648,101
3,chr1,894839,894874,36
4,chr1,1004735,1004904,170
...,...,...,...,...
13203,chrX,154299738,154299918,181
13204,chrY,2709529,2709555,27
13205,chrY,2709577,2709618,42
13206,chrY,2709789,2709805,17


In [185]:
chip200 = chip[(chip['len'] >= 200) & (chip['chr'] == 'chr1')]

In [186]:
chip200['center'] = (chip['end'] + chip['start'])//2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [187]:
chip200

,chr,start,end,len,center
0,chr1,762616,762898,283,762757
1,chr1,840032,840251,220,840141
13,chr1,1370751,1371040,290,1370895
22,chr1,1850719,1851079,361,1850899
27,chr1,3712989,3713250,262,3713119
...,...,...,...,...,...
1282,chr1,246670423,246670623,201,246670523
1286,chr1,247094944,247095359,416,247095151
1287,chr1,247170909,247171130,222,247171019
1288,chr1,249132643,249132996,354,249132819


In [188]:
chip200['my_start'] = chip200['center']-99
chip200['my_end'] = chip200['center'] + 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [189]:
chip200

,chr,start,end,len,center,my_start,my_end
0,chr1,762616,762898,283,762757,762658,762857
1,chr1,840032,840251,220,840141,840042,840241
13,chr1,1370751,1371040,290,1370895,1370796,1370995
22,chr1,1850719,1851079,361,1850899,1850800,1850999
27,chr1,3712989,3713250,262,3713119,3713020,3713219
...,...,...,...,...,...,...,...
1282,chr1,246670423,246670623,201,246670523,246670424,246670623
1286,chr1,247094944,247095359,416,247095151,247095052,247095251
1287,chr1,247170909,247171130,222,247171019,247170920,247171119
1288,chr1,249132643,249132996,354,249132819,249132720,249132919


In [190]:
intersections = dict(zip(chip200['my_start'], chip200['my_end']))

In [191]:
fasta_sequence = SeqIO.parse(input_file,'fasta')
for record in SeqIO.parse(input_file, "fasta"):
 print("%s %i" % (record.id, len(record)))
 name, sequence = record.id, str(record.seq)


chr1 249250621


In [192]:
pos = []
for start, end in intersections.items():
    if set(sequence[start:end+1]).issubset({'A', 'C', 'G', 'T'}):
        pos.append(sequence[start:end+1])

In [193]:
neg = []
for i in range(753):
    x = 1
    while x:
        center = np.random.randint(600000, 200000000)
        if not set(sequence[center-100:center+100]).issubset({'A', 'C', 'G', 'T'}):
            continue
        for start, end in intersections.items():
            if start <= center-100 and center-100 <= end:
                continue
            if start <= center+100 and center+100 <= end:
                continue
        neg.append(sequence[center-100:center+100])
        x = 0


In [194]:
pos

['...']

In [199]:
done_pos = []
for i in pos:
    i = i.replace('A', '0')
    i = i.replace('C', '1')
    i = i.replace('T', '2')
    i = i.replace('G', '3')
    x = list(map(int, list(i)))
    done_pos.append([1] + x)

In [200]:
done_neg = []
for i in neg:
    i = i.replace('A', '0')
    i = i.replace('C', '1')
    i = i.replace('T', '2')
    i = i.replace('G', '3')
    x = list(map(int, list(i)))
    done_neg.append([0] + x)

In [201]:
data = pd.DataFrame(done_pos + done_neg)

In [202]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
0,1,3,2,3,0,1,3,3,0,0,1,1,3,3,0,1,0,1,0,3,0,1,3,1,0,3,0,2,1,2,3,3,1,0,3,1,2,3,0,3,...,3,3,0,1,2,3,1,3,2,3,3,1,2,3,3,3,2,2,2,3,3,1,1,0,1,0,0,0,0,0,3,1,3,3,0,3,3,3,1,0
1,1,3,2,3,3,0,3,0,3,1,3,2,3,3,3,1,1,3,1,0,1,0,3,1,1,2,1,3,3,1,1,3,3,3,0,3,0,3,3,3,...,3,3,1,1,1,2,1,1,3,1,2,1,3,1,3,1,1,3,3,0,3,3,3,1,3,2,3,3,3,1,3,2,3,3,1,1,2,1,3,3
2,1,1,3,1,2,1,0,3,0,3,2,0,0,0,1,1,1,1,3,1,1,1,0,1,1,3,3,3,1,1,1,1,3,1,2,3,3,1,1,3,...,1,0,3,1,0,2,0,0,0,1,1,1,1,3,1,1,1,0,1,2,3,3,3,1,1,1,1,3,1,1,1,0,3,2,3,1,0,0,3,1
3,1,3,0,1,0,1,1,3,1,2,3,1,1,3,1,1,3,1,1,0,2,3,3,1,3,1,1,2,1,0,3,1,2,3,1,2,3,3,1,0,...,3,1,1,2,1,2,1,1,2,3,1,3,2,0,3,0,3,1,1,3,3,3,3,0,3,3,3,3,1,2,3,3,0,3,0,3,1,3,3,1
4,1,2,3,3,0,1,3,3,3,1,3,0,3,3,0,1,3,1,0,2,2,1,3,3,3,3,0,1,0,3,3,3,1,1,1,3,1,3,3,2,...,3,0,0,3,3,3,1,0,3,3,1,3,1,0,1,3,3,1,1,2,1,1,0,3,1,0,3,1,2,3,1,3,3,1,1,0,3,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,3,0,3,1,2,2,1,0,3,3,3,0,0,0,0,2,0,0,2,1,2,3,2,2,0,1,0,3,0,0,3,0,2,0,3,2,1,2,2,...,0,0,2,2,2,3,0,1,0,2,0,2,3,3,2,2,2,2,0,2,0,1,0,1,2,2,1,1,2,2,2,2,2,3,2,0,3,1,2,0
996,0,3,3,3,0,3,1,2,2,3,3,0,3,3,2,2,1,0,3,2,2,3,3,2,1,2,0,3,1,0,2,2,2,1,2,1,0,1,0,0,...,2,0,3,1,0,0,3,1,0,0,1,2,3,3,0,0,3,0,0,3,3,1,2,2,2,1,0,2,2,1,1,2,1,2,1,2,3,0,0,2
997,0,2,3,1,0,0,3,3,3,1,2,1,2,3,3,2,3,3,2,3,3,3,1,2,3,3,3,3,0,1,0,3,1,2,3,3,3,0,1,1,...,1,0,3,3,3,2,0,3,3,3,0,1,1,3,0,3,0,3,3,1,2,2,0,3,3,3,3,1,1,0,3,0,3,2,0,0,3,3,0,1
998,0,1,0,1,0,1,2,1,1,0,1,1,2,1,1,1,0,3,1,2,1,3,0,3,1,1,1,1,3,3,3,3,1,1,1,2,1,2,3,0,...,1,2,3,3,3,0,1,1,2,3,0,3,3,0,3,1,0,1,0,3,3,1,2,3,0,2,3,3,1,1,2,3,0,2,3,1,3,3,1,2


In [207]:
data = data.sample(frac=1).reset_index(drop=True)

**DL - часть**

In [210]:
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

train_size = 800
train = data.iloc[:train_size, :]
test = data.iloc[train_size:, :]
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [220]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.metrics import mean_squared_error

In [221]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [237]:
set_random_seed(67)
train_set = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train)))
train_loader = DataLoader(train_set, batch_size = 20, shuffle = True)

test_set = TensorDataset(torch.Tensor(np.array(X_test)), torch.Tensor(np.array(y_test)))
test_loader = DataLoader(test_set, batch_size = 20, shuffle = True)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
           nn.Dropout(p = 0.7),
           nn.BatchNorm1d(200),
           nn.Linear(200, 200),
           nn.LeakyReLU(),

           nn.Dropout(p = 0.7),
           nn.BatchNorm1d(200),
           nn.Linear(200, 200),
           nn.LeakyReLU(),
           
           nn.Dropout(p = 0.7),
           nn.BatchNorm1d(200),
           nn.Linear(200, 200),
           nn.LeakyReLU(),

           nn.BatchNorm1d(200),
           nn.Linear(200, 1)
        )

    def forward(self, x):
        return self.linear(x)


def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight, gain=1.0)

def init_he(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_uniform_(m.weight, a=0, mode='fan_in', nonlinearity='leaky_relu')

model = Net() #.to(device)
model.apply(init_xavier)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)
criterion = nn.MSELoss()
scheduler = ExponentialLR(optimizer, gamma=0.85)

In [238]:
def train(model, optimizer, criterion, train_loader, test_loader, epochs = 10, scheduler = None):
    model.train()
    for epoch in range(epochs):
        for tracks, labels in tqdm(train_loader):
            #tracks = tracks.to(device)
            #labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(tracks)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if scheduler:
            scheduler.step()
        test(model, criterion)

def test(model, criterion):
    with torch.no_grad():
        model.eval()
        train_pred = model(torch.Tensor(X_train)).detach().numpy()
        test_pred = model(torch.Tensor(X_test)).detach().numpy()
        train_score = mean_squared_error(y_train, train_pred, squared=False)
        test_score = mean_squared_error(y_test, test_pred, squared=False)
        print('Train:', train_score, 'Test:', test_score)


In [239]:
train(model, optimizer, criterion, train_loader, test_loader, 30, scheduler)

  0%|          | 0/40 [00:00<?, ?it/s]

Train: 0.5946280262914088 Test: 0.527211416045364


  0%|          | 0/40 [00:00<?, ?it/s]

Train: 0.400049898579061 Test: 0.39923691716432963


  0%|          | 0/40 [00:00<?, ?it/s]

...

Train: 0.2424759581408707 Test: 0.39690700990710215


  0%|          | 0/40 [00:00<?, ?it/s]

Train: 0.2421101046924312 Test: 0.39810834023623914


  0%|          | 0/40 [00:00<?, ?it/s]

Train: 0.24165174213540747 Test: 0.3984470653550788


Я попроводил несколько экспериментов с разными моделями (полносвязными) и качество везде не очень, и это ожидаемо, и на мой взгляд на это есть две причины:  
1. Очень мало данных, как оказалось мои пересечения гистоновой метки со вторичной структурой белка оказались в среднем не очень длинными, поэтому я брал отрезки +-100, потому что объектов на +-500 было меньше 10. И даже так объектов положительного класса (пересечений) вышло около 300 штук. Поэтому с данными все тяжко.
2. Я взял полносвязную архитектуру для работы с последовательностями, лучше использовать RNN, а еще лучше трансформер, но я пока что не умею такое реализовывать, поэтому сделал только обычный полносвязный перцептрон. 

В будущем в этом пайплайне можно спокойно менять архитектуру на более сложную и вообще проводить разные эксперименты, делая модели для любой пары гистоновой метки и вторичной структуры днк, достаточно просто изменить файлы в начале.